In [2]:
from datasets import load_dataset
from transformers import pipeline
from tqdm import tqdm
import torch
import pandas as pd
from sklearn.metrics import classification_report

In [3]:
dataset = load_dataset('hpe-ai/medical-cases-classification-tutorial')
df = dataset['train'].to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


medical_cases_test.csv:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1724 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/370 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/370 [00:00<?, ? examples/s]

In [4]:
df.head()

,description,transcription,sample_name,medical_specialty,keywords
0,Pacemaker ICD interrogation. Severe nonischem...,"PROCEDURE NOTE: , Pacemaker ICD interrogation....",Pacemaker Interrogation,Cardiovascular / Pulmonary,"cardiovascular / pulmonary, cardiomyopathy, ve..."
1,"Erythema of the right knee and leg, possible s...","PREOPERATIVE DIAGNOSES: , Erythema of the righ...",Aspiration - Knee Joint,Orthopedic,"orthopedic, knee and leg, anterolateral portal..."
2,Left cardiac catheterization with selective ri...,"PREOPERATIVE DIAGNOSIS: , Post infarct angina....",Cardiac Cath & Selective Coronary Angiography,Cardiovascular / Pulmonary,"cardiovascular / pulmonary, selective, angiogr..."
3,Patient with a history of coronary artery dise...,"REASON FOR VISIT: , Acute kidney failure.,HIST...",Acute Kidney Failure,Nephrology,None
4,Cardiac evaluation and treatment in a patient ...,"REASON FOR REFERRAL: , Cardiac evaluation and ...",Cardiac Consultation - 6,Cardiovascular / Pulmonary,None


In [5]:
def simplify_specialty(specialty):
    return specialty.split("/")[0].split("-")[0].strip()

df['medical_specialty'] = df['medical_specialty'].apply(simplify_specialty)

In [1]:
def clean_prediction(prediction):
    prediction = prediction.strip().split("\n")[0]  # Get only the first line
    prediction = prediction.replace("-", " ").replace(",", " ")
    words = prediction.split()

    # Try to find the first matching specialty from your dataset
    for word in words:
        for specialty in df['medical_specialty'].unique():
            if word.lower() in specialty.lower() or specialty.lower() in word.lower():
                return specialty

    # print(prediction)
    print(f'Wrong word: {prediction}')
    return "Unknown"

In [7]:
def classify_medical_specialty(pipe, current_sample, df, all_categories, n_shot=0):

    few_shot_examples = []
    for category in all_categories:
        class_samples = df[df['medical_specialty'] == category]
        sampled = class_samples.sample(n=min(n_shot, len(class_samples)), random_state=42)
        few_shot_examples.extend(sampled.to_dict(orient='records'))

    # print(few_shot_examples)
    prompt = f"""You are a medical assistant. Given the description, your job is to classify each medical case into ONLY ONE of the following specialties:
    {', '.join(all_categories)}

      Respond with ONLY the name of the correct medical specialty from the given list only (no word should be out of list).
    """

    for i, ex in enumerate(few_shot_examples):
        prompt += f"""
          Example {i+1}:
          Description: {ex['description']}
          medical specialty': {ex['medical_specialty']}
        """

    prompt += f"""

    Now, classify the following:

    Description: {current_sample['description']}

    medical specialty:"""

    output = pipe(prompt)[0]["generated_text"]

    # Clean the output to match known categories
    # return clean_prediction(output)
    return clean_prediction((output.split('medical specialty:')[1]))
    # return output
    # return clean_prediction(prediction)


In [8]:
from transformers import AutoTokenizer, pipeline

model_id = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    device_map="cuda",
    max_new_tokens=15,
    pad_token_id=tokenizer.eos_token_id  # Avoid warning
)

Device set to use cuda


In [9]:
def n_shot_classify(n_shot):
  pred = []
  for i in tqdm(range(len(df))):

    row = df.iloc[i]

    output = classify_medical_specialty(pipe, row, df, df['medical_specialty'].unique(), n_shot=n_shot)

    torch.cuda.empty_cache()

    pred.append(output)
  return pred

In [11]:
pred_zero_shot = n_shot_classify(0)

  0%|          | 1/1724 [00:02<1:23:21,  2.90s/it]

Wrong word: Cardiology


  0%|          | 2/1724 [00:04<56:58,  1.99s/it]  

Wrong word: ```


  0%|          | 7/1724 [00:11<43:55,  1.53s/it]

...

Wrong word: Description:


100%|█████████▉| 1716/1724 [38:21<00:09,  1.14s/it]

Wrong word: ```


100%|██████████| 1724/1724 [38:32<00:00,  1.34s/it]

Wrong word: Cardiology


In [12]:
print(classification_report(pred_zero_shot,df['medical_specialty']))

                  precision    recall  f1-score   support

  Cardiovascular       0.54      0.32      0.40       902
             ENT       0.04      0.05      0.04        37
Gastroenterology       0.00      0.00      0.00         1
      Hematology       0.00      0.00      0.00        16
      Nephrology       0.07      0.43      0.12         7
       Neurology       0.01      1.00      0.01         1
    Neurosurgery       0.00      0.00      0.00         1
      Obstetrics       0.01      0.50      0.02         2
   Ophthalmology       0.02      1.00      0.04         1
      Orthopedic       0.05      0.64      0.09        22
      Pediatrics       0.02      0.50      0.04         2
      Psychiatry       0.02      1.00      0.04         1
       Radiology       0.00      0.00      0.00         0
         Unknown       0.00      0.00      0.00       731

        accuracy                           0.18      1724
       macro avg       0.06      0.39      0.06      1724
    weighted

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [12]:
pred_three_shot = n_shot_classify(3)

  0%|          | 0/1724 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print(classification_report(pred_three_shot,df['medical_specialty']))

In [ ]:
pred_five_shot = n_shot_classify(5)

In [ ]:
print(classification_report(pred_five_shot,df['medical_specialty']))